In [ ]:
# Importation des bases
import pandas as pd
import functions as fc

dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

In [ ]:
etab = dict_data['etablissements']
etab_not_null = etab.dropna(subset = ['code_epsg'])
etab_null = etab.loc[etab['code_epsg'].isnull()]
gb = etab_not_null.groupby("code_epsg")
gb = [gb.get_group(x) for x in gb.groups]

In [ ]:
temp = [
    fc.transform_wgs84(
        gb[idx],
        gb[idx]['code_epsg'].iloc[0]
    ) for idx in range(len(gb)) 
]
temp = pd.concat(
    temp
)
etab = pd.concat(
    [temp, etab_null]
)

In [ ]:
df = etab.copy()
df = df.rename({'numero_siret': "numero_siret_true"}, axis = 1)
df["code_apet"] = df["code_ape"].str[:4]

requete_type = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{ "match": {{ "rs_denom":   "{nom_etablissement}" }}}}
      ],
      "filter": [
        {{ "match":  {{ "adr_et_post": "{code_postal}" }}}},
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''




# Il est nécessaire de spécifier l'index associé à chaque requête
header = '{"index" : "sirus_2020"}'

multiple_requetes = ""

# On itère sur le dataframe d'établissements polluants pour ajouter une requête spécifique à chacun d'entre eux
for index, row in df.loc[:,['nom_etablissement',  'code_postal', 'code_apet']].iterrows():
    
    multiple_requetes+= header
    multiple_requetes+= '\n'
    multiple_requetes+= requete_type.format_map(row).replace("\n","")
    multiple_requetes+= '\n'



res = fc.es.msearch(body = multiple_requetes, max_concurrent_searches = 500)

temp = pd.json_normalize(res,record_path=["responses"])

out_elastic = temp.apply(
        lambda l: fc.get_product_echo(l['hits.hits']),
        axis=1, result_type="expand")
out_elastic.head(1)

In [ ]:
pd.concat(
    [df, out_elastic]
    )

## Les émissions de CO2

On va commencer par se focaliser sur les gros émetteurs de CO2

In [ ]:
emissions = dict_data['emissions']
emissions_co2 = emissions.loc[emissions['polluant'].str.contains("CO2")]
emissions_co2 = emissions_co2.loc[emissions_co2['polluant'].str.contains('total')]
emissions_co2.head(1)

In [ ]:
emissions_co2['quantite'].div(1000).plot.hist(logx = True)

In [ ]:
dict_data.head(1)

In [ ]:
dict_data['emissions']["annee_emission"].value_counts()

In [ ]:
dict_data['emissions']["milieu"].value_counts()

In [ ]:
dechets_dang = dict_data['Trait_dechets_dangereux']

In [ ]:
dict_data['etablissements']['identifiant'].nunique()

In [ ]:
dict_data['etablissements'].head(1)

In [ ]:
dict_data